# RNA-Seq from scratch - Kallisto

In this notebook we setup an [Atmosphere](https://atmo.cyverse.org/application) linux virtual machine so that we can analzye RNA-Seq data using [Kallisto](https://pachterlab.github.io/kallisto/). 



## Software Installations

First, let's install [miniconda](https://conda.io/miniconda.html). This is a software package that will help us easily install other software we might need. Although in this example we will be installing miniconda on a Linux machine, you can use the miniconda link above to download and install for your operating system (Windows, Mac, etc.)

### Install Miniconda

First let verify that we are in a directory that would be good for installing this software

In [ ]:
pwd

We are in our home directory, and for the purpose of this tutorial, this will be fine. Let's download the installation file using `wegt`:

In [ ]:
wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh && ls

Notice in the above cell we used `wget` to download `Miniconda3-latest-Linux-x86_64.sh` and also used the `ls` command to verify we downloaded the file. Now let's install it. 

In [ ]:
bash Miniconda3-latest-Linux-x86_64.sh -b -p $HOME/miniconda

Next, let's tell the computer where miniconda is by exporting to path:

In [ ]:
export PATH="$HOME/miniconda/bin:$PATH"

Finally, we will configure conda so that it is aware of [bioconda](https://bioconda.github.io/index.html) packages

In [ ]:
conda config --add channels defaults
conda config --add channels conda-forge
conda config --add channels bioconda

### Install Kallisto using Bioconda

Next, we can install Kallisto (one of [many packages](https://bioconda.github.io/recipes.html)) available on bioconda. 

In [ ]:
# we use the -y option so that we automatically accept all prompts
conda install -y kallisto

Now, let's verify that kallisto is installed, and check the version by running the kallisto command with no arguments. 

In [ ]:
kallisto

### Install SRA Tools and import data from SRA (optional)

We need to obtain some test data to analyze. To do so, we will import some data from the [NCBI SRA](https://www.ncbi.nlm.nih.gov/sra). The data we are working with is and Arabidopsis dataset described [here](https://www.ncbi.nlm.nih.gov/bioproject/PRJNA79729/). 

First we need to get the list of accessings (sequencing runs) which is available for download here: https://www.ncbi.nlm.nih.gov/bioproject/PRJNA79729/. We are looking for the `SraRunTable.txtx` file. which can be downloaded here: https://www.ncbi.nlm.nih.gov/Traces/study/?acc=SRP003234. We have provided it on this instance:

In [ ]:
head -n 4 $HOME/kallisto-rnaseq-jupyter/required_files/SraRunTable.txt

This is quite hard to read, but we need the `Run` column to download read data. 

In [ ]:
cut -f7 $HOME/kallisto-rnaseq-jupyter/required_files/SraRunTable.txt

Now, let's do two things. We are going to use the [SRA Toolkit](https://trace.ncbi.nlm.nih.gov/Traces/sra/sra.cgi?view=toolkit_doc) to import the files we need from the SRA. Rather than do 19 downloads one-by-one, we can take this list of accessions and use a while loop to do the import. Also, for the purpose of this demo we won't download all 19 files but just take the first 1. Let's make a file that just has the first to entries from our `SraRunTable.txt`. 

In [ ]:
cut -f7 $HOME/kallisto-rnaseq-jupyter/required_files/SraRunTable.txt |grep -v "Run"| head -n1 > sample_runs.txt

Let's look at the file we created. In the command above we just created a file that has a single SRA accession, but in principle you could do without the last pipe `| head -n1` and simply save all the accession IDs to `sample_runs.txt`

In [ ]:
cat sample_runs.txt

let's use bioconda to install the Sra Toolkit and update our path

In [ ]:
conda install -y sra-tools && export PATH="$HOME/miniconda/bin:$PATH"

**(Optional - we have this data on the image so this can be skipped)** 

We will use a while loop to read the list of run names and import them from ncbi. There are some additional options we can use to import the data more quickly, but for now we will just use the simplest options. 

**(This takes about 5 minutes to import the data)**

In [ ]:
while read line; do prefetch $line; done<sample_runs.txt

Lets move these files into a more convenient location

In [ ]:
mkdir $HOME/raw_data && mv $HOME/ncbi/public/sra/*.sra $HOME/raw_data

We now have two sra files in the `raw_data` directory

In [ ]:
ls $HOME/raw_data

**(OPTIONAL)** We now need to use another tool to convert these files into fastq format. We will covert them to a compressed (fastq.gz) format which can be directly used by Kallisto. This will take ~5 minutes per file (good thing we are only doing one)

In [ ]:
cd $HOME/raw_data && for file in *.sra; do fastq-dump --gzip $file; done

All of the data we need is on CyVerse, so we will use iCommands to import the rest of the dataset (much faster). While we haven't provided data on using iCommands and CyVerse Data Store, you can learn more about them at https://cyverse-data-store-guide.readthedocs-hosted.com/en/latest/

In [ ]:
rm $HOME/raw_data/*.sra # Get rid of the sra file we won't use 
mv $HOME/fastq_files $HOME/raw_data

We should now have our 19 fastq files

In [ ]:
ls $HOME/raw_data/fastq_files

## Setup and run Kallisto

Next we will get some reference transcriptome data and start the process of running Kallisto

### Import transcriptome data

First, we will download the Arabdiopsis transcriptome data from [Ensemble](ftp://ftp.ensemblgenomes.org/pub/plants/release-39/fasta/arabidopsis_thaliana/cdna/)

In [ ]:
wget ftp://ftp.ensemblgenomes.org/pub/plants/release-39/fasta/arabidopsis_thaliana/cdna/Arabidopsis_thaliana.TAIR10.cdna.all.fa.gz

Verify the the checksum of the downloaded file with the [publushed sum](ftp://ftp.ensemblgenomes.org/pub/plants/release-39/fasta/arabidopsis_thaliana/cdna/CHECKSUMS)

In [ ]:
sum Arabidopsis_thaliana.TAIR10.cdna.all.fa.gz

Let's also organize our downloaded data

In [ ]:
mkdir transcriptome && mv Arabidopsis_thaliana.TAIR10.cdna.all.fa.gz transcriptome

### Index transcriptome

We will now use Kallisto's indexing function to prepare the transcriptome for analysis. First let's organize our files:

In [ ]:
mkdir $HOME/analysis

Next run the indexing command. This prepares the transcriptome so that we can peudoalign reads to it. 

In [ ]:
kallisto index --index="athalianaTAIR10_index" $HOME/raw_data/transcriptome/Arabidopsis_thaliana.TAIR10.cdna.all.fa.gz

We now have a transcriptome index which can now be used for pseudoalignment, we'll move it into the transcriptome folder:

In [ ]:
mv athalianaTAIR10_index transcriptome/

### Quantify reads

In this final step, we will run Kallisto on all of our files to quantify the reads. We will write a for loop to do this. 

In [ ]:
pwd
ls

All instructions for the commands we are using are in the Kallisto manual: https://pachterlab.github.io/kallisto/manual. *If needed, the results for this are located on the CyVerse Data commons at (/iplant/home/shared/cyverse_training/datasets/PRJNA79729/kallisto_quantified) and on the Amazon AMI in the dcuser home directory in the `.quantfied` folder.*



In [ ]:
cd $HOME/raw_data/fastq_files
for file in *.fastq.gz; do output="${file%.*.*}"_quant; kallisto quant\
 --single\
 --index=$HOME/raw_data/transcriptome/athalianaTAIR10_index\
 --single\
 --bootstrap-samples=25\
 --fragment-length=38\
 --sd=1\
 --output-dir=$output\
 $file; done

Finally, we can move our results folders into our analysis folder:

In [ ]:
mv $HOME/raw_data/fastq_files/*/ $HOME/analysis 
ls $HOME/analysis